In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-bert-cv-def

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  40.45kB
Step 1/32 : FROM nvidia/cuda:10.0-cudnn7-devel-ubuntu16.04
10.0-cudnn7-devel-ubuntu16.04: Pulling from nvidia/cuda
be8ec4e48d7f: Pulling fs layer
33b8b485aff0: Pulling fs layer
d887158cc58c: Pulling fs layer
05895bb28c18: Pulling fs layer
84ba571a9830: Pulling fs layer
015c2871e963: Pulling fs layer
f554ab5a3b6a: Pulling fs layer
b6c7436100b9: Pulling fs layer
796fba3f2ca9: Pulling fs layer
0f65bbc1e210: Pulling fs layer
fb3b3401f4c6: Pulling fs layer
ec79cf45e3fe: Pulling fs layer
84ba571a9830: Waiting
015c2871e963: Waiting
f554ab5a3b6a: Waiting
b6c7436100b9: Waiting
796fba3f2ca9: Waiting
0f65bbc1e210: Waiting
fb3b3401f4c6: Waiting
ec79cf45e3fe: Waiting
05895bb28c18: Waiting
33b8b485aff0: Verifying Checksum
33b8b485aff0: Download complete
d887158cc58c: Verifying Checksum
d887158cc58c: Download complete
05895bb28c18: Verifying Checksum
05895bb28c18: Download complete
84ba571a9830: Verifying Checksum
84ba571a9830: Download 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 148 ms, sys: 55.1 ms, total: 203 ms
Wall time: 23min 21s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_bert_cv_def'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-bert-cv-def:latest'.format(account, region)
vdok3bert = sage.estimator.Estimator(image,
                       role, 1, 'ml.p3.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3bert.fit(data_location)

2021-03-13 01:47:40 Starting - Starting the training job...
2021-03-13 01:48:02 Starting - Launching requested ML instancesProfilerReport-1615600059: InProgress
......
2021-03-13 01:49:03 Starting - Preparing the instances for training......
2021-03-13 01:50:08 Downloading - Downloading input data
2021-03-13 01:50:08 Training - Downloading the training image......................./.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Starting the training.

2021-03-13 01:54:04 Training - Training image download completed. Training in progress.bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type

In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3bert.deploy(1, 'ml.p3.2xlarge', serializer=csv_serializer)

-------------------!

In [7]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Score_Class,Definition-Score
0,0
1,1
1,1
2,2



In [8]:
sess.delete_endpoint(predictor.endpoint)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3bert.transformer(instance_count=1,
                               instance_type='ml.p3.2xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

In [10]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

..............................................Starting the inference server with 8 workers.
2021/03/13 02:34:21 [crit] 42#42: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [13/Mar/2021:02:34:21 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
2021/03/13 02:34:21 [crit] 42#42: *3 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [13/Mar/2021:02:34:21 +0000] "GET /ping HTTP/1.1" 502 182 "-" "Go-http-client/1.1"
2021/03/13 02:34:21 [crit] 42#42: *5 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting